In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, State
import random

file_path = r'G:\Unidades compartidas\ADMINISTRACIÓN\Estadísticas\Examenes_oficiales.csv'

# Probar con delimitador de punto y coma
df2 = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=';')

Temario = ['Cuerpo', 'Bloque', 'Tema', 'Epígrafe', 'OEP', 'Modalidad', 'Pregunta']
ExOf = df2[Temario]
ExOf.loc[:, 'Num'] = 1

app = Dash(__name__)

# Definir una lista de colores en tonos pastel
pastel_colors = px.colors.qualitative.Pastel


# Layout de la aplicación
app.layout = html.Div([
    html.H1("Preguntas de Exámenes Oficiales"),
    html.H3("Haz clic en los recuadros para obtener más información"),

    # Botón para volver al inicio
    html.Button("Volver al inicio", id="reset-button", n_clicks=0),
    
    # Componente para mostrar el TreeMap
    dcc.Graph(id='treemap'),
    
    # Componente para mostrar la tabla
    dcc.Graph(id='table')
])

# Callback para actualizar solo el TreeMap cuando se hace clic en "Volver al inicio"
@app.callback(
    Output('treemap', 'figure'),
    Input('reset-button', 'n_clicks')
)
def reset_treemap(n_clicks):
    # Crear la figura del TreeMap sin filtros
    treemap_fig = px.treemap(
        ExOf,
        path=[px.Constant("Preguntas de Exámenes Oficiales"), 'Cuerpo', 'Modalidad', 'Bloque', 'Tema', 'Epígrafe', 'OEP'],
        values='Num',
        maxdepth=3,
        #color = 'Bloque',
        color_discrete_sequence = pastel_colors,
        custom_data=['Num']
    ).update_traces(
        hovertemplate='<b>%{label}</b><br>Nº de preguntas: %{value}<extra></extra>',
        texttemplate='%{label}<br><span style="font-size:10px">Nº de preguntas: %{value}</span>',
        textposition='top left',
        textfont=dict(
            family="Arial",
            size=14,
        )
    ).update_layout(
        height=400,
        margin=dict(t=50, l=15, r=15, b=15)
    )
    return treemap_fig

# Callback para actualizar la tabla solo cuando se selecciona un cuadro del TreeMap
@app.callback(
    Output('table', 'figure'),
    Input('treemap', 'clickData')
)
def update_table(clickData):
    # Si no se ha hecho clic, mostrar todas las preguntas
    if clickData is None:
        filtered_df = ExOf
    else:
        # Inicializar el filtro acumulativo basado en los niveles seleccionados
        filters = {
            'Cuerpo': None,
            'Modalidad': None,
            'Bloque': None,
            'Tema': None,
            'Epígrafe': None,
            'OEP': None
        }

        # Obtener el camino de selección actual en el TreeMap
        path = clickData['points'][0]['id'].split("/")[1:]  # Excluir el primer elemento ("Preguntas de Exámenes Oficiales")

        # Asignar valores seleccionados en el filtro progresivo
        for i, level in enumerate(filters.keys()):
            if i < len(path):
                filters[level] = path[i]
        
        # Aplicar filtros acumulativos al DataFrame
        filtered_df = ExOf.copy()
        for col, value in filters.items():
            if value is not None:
                filtered_df = filtered_df[filtered_df[col] == value]

    # Mostrar solo una vez cada valor de 'Pregunta' en la tabla, sin eliminar del DataFrame original
    filtered_df_unique = filtered_df.drop_duplicates(subset=['Pregunta'])
    
    # Crear la tabla con el DataFrame filtrado
    table_fig = go.Figure(data=[go.Table(
        header=dict(values=['Tema', 'Epígrafe', 'OEP', 'Pregunta']),
        cells=dict(values=[filtered_df_unique['Tema'], filtered_df_unique['Epígrafe'], filtered_df_unique['OEP'], filtered_df_unique['Pregunta']])
    )])
    
    return table_fig


# Correr la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18768\1810725534.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

